In [11]:
!pip install -q -U google-generativeai
!pip install -q googlesearch-Python

In [2]:
import google.generativeai as genai
from google.colab import userdata # Importa a função para acessar secrets

# Carrega a chave da API do Colab Secrets
try:
    API_KEY = userdata.get('API_KEY')
    genai.configure(api_key = API_KEY)
    print("API Key configurada com sucesso!")

except Exception as e:
    print(f"Erro ao configurar a API Key. Verifique se ela está salva nos Secrets do Cloab com o nome 'API_KEY. Erro: {e}")

API Key configurada com sucesso!


In [17]:
# Instancia o modelo que vamos usar

model = genai.GenerativeModel(
    model_name = 'gemini-2.0-flash',
    tools = tools_description # <-- Passamos TODAS as ferramentas AQUI!
    )
print(f"Modelo '{model.model_name}' instanciado com sucesso!")

Modelo 'models/gemini-2.0-flash' instanciado com sucesso!


In [21]:
# Cria lista na memória
lista_lembretes = []

In [15]:
# Cria função adicionar lembrete
def adicionar_lembrete_func(tarefa: str, horario: str, data: str = None):
    """Adiciona um lembrete para o paciente sobre uma tarefa médica ou compromisso."""
    lembrete_str = f"Lembrete: {tarefa} às {horario}"
    if data:
        lembrete_str += f" em {data}"
    # Adiciona à lista
    lista_lembretes.append(lembrete_str)
    print(f"\nDEBUG: Função 'adicionar_lembrete_func' executada. Lembrete: '{lembrete_str}'")

    # Retorna uma confirmação para a IA usar na resposta final
    return f"Lembrete para '{tarefa}' anotado com sucesso."

    # --- Ferramenta de busca na Internet ---
    from googlesearch import search  # Importa a função de busca

def buscar_na_internet_func(query: str):
      """Busca informações na internet usando a consulta fornecida."""
      print(f"\nDebug: Função 'buscar_na_internet_func' executada com query: '{query}'")
      try:
        # Realiza a busca (linita para os 5 primeiros resultados, timeout de 5 segundos)
        # O stop=5 significa para após encontrar 5 resultados.
        search_results = list(search(query, num_results = 5, timeout = 5))
        if not search_results:
          return"Nenhum resultado encontrado na internet para sua busca."

     # Formata os resultados para enviar para a IA
        # Vamos enviar uma lista simples de URLs e talvez um snippet se a biblioteca fornecer
        formatted_results = "Resultados da busca:\n"
        for i, result in enumerate(search_results):
             # A biblioteca googlesearch retorna apenas URLs, não snippets diretamente em todos os casos simples
             formatted_results += f"{i+1}. {result}\n"

        # Limita o tamanho do resultado para não estourar o limite de tokens da IA
        if len(formatted_results) > 1500:
             formatted_results = formatted_results[:1500] + "..." # Trunca se for muito longo


        return formatted_results

      except Exception as e:
        print(f"DEBUG: Erro ao executar busca na internet: {e}")
        return f"Ocorreu um erro ao buscar na internet: {e}"

# --- ATUALIZAR dictionary tools_available ---
# Mapeia os nomes das ferramentas (como a IA vai "pedir") para as funções Python reais
tools_available = {
    "adicionar_lembrete": adicionar_lembrete_func,
    "buscar_na_internet": buscar_na_internet_func, # Adicionamos a nova ferramenta aqui!
}

print("Funções (ferramentas) definidas.")


Funções (ferramentas) definidas.


In [16]:
# Descrição das Ferramentas para passar para a API do Gemini
tools_description = [
    {
        "function_declarations": [
            {
                "name": "adicionar_lembrete", # O nome EXATO da chave no dictionary tools_available (em Célula 4)
                "description": "Adiciona um lembrete para o paciente sobre uma tarefa médica ou compromisso, incluindo o que lembrar, horário e data (se especificada).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "tarefa": {
                            "type": "string",
                            "description": "A descrição clara e concisa da tarefa a ser lembrada (ex: tomar remédio, consulta com nefrologista, fazer exame de sangue)."
                        },
                        "horario": {
                            "type": "string",
                            "description": "O horário específico do lembrete na linguagem do usuário (ex: 8 da manhã, 14:30, meio-dia). Tente extrair o mais preciso possível."
                        },
                         "data": {
                            "type": "string",
                            "description": "A data do lembrete se for para um dia específico diferente de hoje (ex: amanhã, segunda-feira, 25 de dezembro). Opcional.",
                            "nullable": True # Indica que este parâmetro é opcional
                        }
                    },
                    "required": ["tarefa", "horario"] # Campos obrigatórios que a IA DEVE tentar extrair
                }
            },


            # --- NOVA DESCRIÇÃO DE FERRAMENTA: Buscar na Internet ---
            {
                "name": "buscar_na_internet", # Nome EXATO da chave em tools_available
                "description": "Busca informações gerais na internet sobre um tópico específico. Use para responder perguntas sobre eventos recentes, fatos específicos, definições, ou qualquer coisa que não esteja no conhecimento geral do assistente ou exija dados atualizados.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "A consulta de busca clara e concisa a ser usada para encontrar a informação relevante."
                        }
                    },
                    "required": ["query"] # A IA DEVE fornecer uma consulta para a busca
                }
            }
            # Adicionar descrições de outras ferramentas aqui
        ]
    }
]

print("Descrição das ferramentas (agora incluindo busca) criada.")

Descrição das ferramentas (agora incluindo busca) criada.


In [7]:
# Instrução do Sistema (persona geral)
system_instruction_text = """Você é um assistente pessoal prestativo e informativo para pacientes que passaram por um transpalnte de órgão. Sua principal funçao é fornecer informações gerais sobre cuidados pós-transpalnte e ajudar a gerenciar lembretes. Minhas informações NÂO substituem aconselhamento médico profissional."""


In [8]:
# Iniciamos o chat passando a instrução do sistema no history e as descrições das ferramentas.

# Instrução do Sistema (persona geral) - Certifique-se que esta variável foi definida na célula anterior
system_instruction_text = """Você é um assistente pessoal prestativo e informativo para pacientes que passaram por um transplante de órgão. Sua principal função é fornecer informações gerais sobre cuidados pós-transplante e ajudar a gerenciar lembretes.

Regras Importantes:
1. Sempre reforce que suas informações NÃO substituem o aconselhamento médico profissional.
2. Use as ferramentas disponíveis quando apropriado para ajudar o paciente (por exemplo, use a ferramenta de lembrete quando for solicitado a anotar algo).
3. Se não puder ajudar com as ferramentas disponíveis ou com informação geral, diga isso educadamente.
"""

# Certifique-se que a lista tools_description foi definida na Célula 5
# tools_description = [...]

try:
    # Corrigido: model.start_chat (com 's' e 't')
    # Corrigido: Indentação de tools_config
    chat = model.start_chat(
        history=[{"role": "system", "parts": [system_instruction_text]}],
        tools=tools_description,
        tools_config={"function_calling_config": "ANY"} # Alinhado com os outros parâmetros
    )
    print("\nChat iniciado com instrução do sistema e ferramentas.")
except Exception as e:
     print(f"Erro ao iniciar o chat com ferramentas. Detalhe: {e}")
     # Se der erro aqui, pode ser problema com a API ou o nome das ferramentas/parametros na descrição (menos provável agora).

Erro ao iniciar o chat com ferramentas. Detalhe: GenerativeModel.start_chat() got an unexpected keyword argument 'tools'


In [23]:
# Célula 7: Loop Principal de Conversa com Lógica de Chamada de Função (Incluindo Busca)
print("\n--- Assistente Pronto! Digite 'sair' para encerrar. ---")

while True: # Cria um loop infinito para a conversa
    user_input = input("Você: ") # Pede a entrada do usuário

    if user_input.lower() == "sair": # Adiciona uma forma de sair do chat
        print("Assistente: Encerrando o chat. Cuide-se!")
        # Opcional: Mostrar lembretes agendados ao sair
        if lista_lembretes:
             print("\nLembretes agendados:")
             for l in lista_lembretes:
                 print(f"- {l}")
        break # Sai do loop

    try:
        # Envia a mensagem do usuário para o chat (a IA pode responder com texto OU uma chamada de função)
        response = chat.send_message(user_input)

        # --- Lógica de Orquestração: Verifica a resposta do modelo ---

        # 1. Verifica se a IA pediu para executar uma função
        if response.function_calls:
            print("DEBUG: IA solicitou chamada(s) de função.")

            # Executa cada chamada de função que a IA solicitou (geralmente será uma por vez)
            for fc in response.function_calls:
                tool_name = fc.name # Nome da função que a IA quer chamar (ex: "adicionar_lembrete", "buscar_na_internet")
                tool_args = fc.args # Argumentos que a IA extraiu (dicionário, ex: {'tarefa': '...'} ou {'query': '...'})

                if tool_name in tools_available:
                    print(f"DEBUG: Executando ferramenta: '{tool_name}' com args: {tool_args}")
                    try:
                        # --- Executar a Ferramenta Real (sua função Python) ---
                        # Encontra a função Python correspondente e a chama
                        # Usamos **tool_args para passar os argumentos do dicionário para os parâmetros da função
                        tool_result = tools_available[tool_name](**tool_args)

                        print(f"DEBUG: Resultado da ferramenta '{tool_name}': '{tool_result}'")

                        # --- Enviar o Resultado da Ferramenta de Volta para a IA ---
                        # Isso é CRUCIAL. Permite que a IA saiba que a ferramenta rodou e qual foi o resultado,
                        # para então gerar a resposta final para o usuário.
                        response_from_tool = chat.send_message(
                            {"role": "tool", # Indica que esta mensagem é a resposta DA FERRAMENTA
                             "parts": [{"functionResponse": {"name": tool_name, "response": {"content": tool_result}}}]} # Formato esperado pela API
                        )

                        # --- Obter a Resposta Final da IA para o Usuário ---
                        # A resposta final (que usou o resultado da ferramenta para formular a resposta)
                        if response_from_tool.text:
                             print(f"Assistente: {response_from_tool.text}")
                        else:
                             print("Assistente: OK. Ação realizada ou informação obtida.") # Confirmação genérica

                    except Exception as ex_func:
                        error_message = f"Erro ao executar a ferramenta '{tool_name}'. Detalhe: {ex_func}"
                        print(f"DEBUG: {error_message}")
                        print(f"Assistente: Desculpe, tive um problema ao tentar executar a ação solicitada ({tool_name}).")


                else:
                    # 2. A IA pediu uma ferramenta que definimos, mas o nome não bate com tools_available? (Raro)
                    print(f"DEBUG: IA solicitou ferramenta desconhecida em tools_available: {tool_name}")
                    print(f"Assistente: Desculpe, não consigo realizar essa ação solicitada ({tool_name}).")


        # 3. Se a IA NÃO pediu função, ela respondeu apenas com texto direto
        elif response.text:
            print("DEBUG: IA respondeu apenas com texto.")
            print(f"Assistente: {response.text}")

        # 4. Outro tipo de resposta ou resposta vazia
        else:
             print("DEBUG: Resposta da IA vazia ou inesperada.")
             print("Assistente: Não consegui obter uma resposta válida no momento.")


    except Exception as e:
        print(f"DEBUG: Erro geral no processamento da resposta: {e}")
        print(f"Assistente: Ocorreu um erro ao processar sua mensagem. Por favor, tente novamente. Erro: {e}")

# --- Fim do Loop ---


--- Assistente Pronto! Digite 'sair' para encerrar. ---
Você: rejeição
DEBUG: Erro geral no processamento da resposta: name 'chat' is not defined
Assistente: Ocorreu um erro ao processar sua mensagem. Por favor, tente novamente. Erro: name 'chat' is not defined
Você: sair
Assistente: Encerrando o chat. Cuide-se!
